# SQL - задачи

## Введение

Нам необходимо проанализировать базу данных в недавно купленном сервисе по чтению книг по подписке. Анализ поможет сформулировать ценностное предложение для нового продукта, а так же перспективы для обновлений книг в наших базах.
Нам предстоит
- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так ми исключим из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Изучение данных

Для начала импортируем библиотеки и подключимся к базе данных.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# устанавливаем параметры
db_config = {'user': 'user', # имя пользователя
             'pwd': 'user_pwd', # пароль
             'host': 'host',
             'port': 0000, # порт подключения
             'db': 'db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                 db_config['pwd'],
                                                 db_config['host'],
                                                 db_config['port'],
                                                 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Напишем функцию для вывода первичной информации о данных.

In [ ]:
def startinfo(data):
  
    display(data.head(5))
    data.info()
    print((f'Дубликатов - "{data.duplicated().sum()}"'))

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

In [ ]:
query = '''
           SELECT * 
           FROM books
           
        '''
data_books = pd.io.sql.read_sql(query, con = engine) 
startinfo(data_books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB
Дубликатов - "0"


Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

In [ ]:
query = '''
           SELECT * 
           FROM authors
           
        '''
data_authors = pd.io.sql.read_sql(query, con = engine) 
startinfo(data_authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB
Дубликатов - "0"


**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

In [ ]:
query = '''
           SELECT * 
           FROM publishers
        '''
data_publishers = pd.io.sql.read_sql(query, con = engine) 
startinfo(data_publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB
Дубликатов - "0"


**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

In [ ]:
query = '''
           SELECT * 
           FROM ratings
        '''
data_rating = pd.io.sql.read_sql(query, con = engine) 
startinfo(data_rating)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB
Дубликатов - "0"


**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [ ]:
query = '''
           SELECT * 
           FROM reviews
        '''
data_review = pd.io.sql.read_sql(query, con = engine)
startinfo(data_review)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB
Дубликатов - "0"


Мы загрузили и посмотрели все предоставленные данные. В них нет дубликатов и пропущенных.

## Задачи

 **Задача 1**
- Посчитать, сколько книг вышло после 1 января 2000 года

In [ ]:

query = '''
           SELECT COUNT(book_id) 
           FROM books
           WHERE publication_date > '2000-01-01'; 
        '''
pd.io.sql.read_sql(query, con = engine) 

,count
0,819


После 1 янваля 2000 года вышло 819 книг.

**Задача 2** 
- Для каждой книги посчитать количество обзоров и среднюю оценку

In [ ]:
query = '''
           SELECT b.title AS name,
                  b.book_id AS id,
                  rv.count_review AS count_review,
                  rt.avg_rating AS avg_rating
                  
           FROM books AS b 
           LEFT JOIN 
                (SELECT book_id,
                        AVG(rating) AS avg_rating
                 FROM ratings
                 GROUP BY book_id) AS rt ON rt.book_id = b.book_id
           LEFT JOIN 
                (SELECT book_id,
                        COUNT(review_id) AS count_review
           FROM reviews
           GROUP BY book_id) AS rv ON rv.book_id = b.book_id
                
           
        '''
pd.io.sql.read_sql(query, con = engine) 

,name,id,count_review,avg_rating
0,The Body in the Library (Miss Marple #3),652,2.0,4.500000
1,Galápagos,273,2.0,4.500000
2,A Tree Grows in Brooklyn,51,5.0,4.250000
3,Undaunted Courage: The Pioneering First Missio...,951,2.0,4.000000
4,The Prophet,839,4.0,4.285714
...,...,...,...,...
995,Alice in Wonderland,64,4.0,4.230769
996,A Woman of Substance (Emma Harte Saga #1),55,2.0,5.000000
997,Christine,148,3.0,3.428571
998,The Magicians' Guild (Black Magician Trilogy #1),790,2.0,3.500000


**Задача 3** 
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [ ]:
query = '''
           SELECT  p.publisher,
                   COUNT(p.publisher)
           FROM publishers AS p
           iNNER JOIN  (SELECT *
                        FROM books
                        WHERE num_pages > 50) AS b ON  b.publisher_id	 = p.publisher_id
           GROUP BY p.publisher
           ORDER BY COUNT(p.publisher) DESC
           LIMIT 1
        '''
pd.io.sql.read_sql(query, con = engine) 

,publisher,count
0,Penguin Books,42


Британское издательство *Penguin Books* выпустило наибольшее число книг толще 50 страниц.

**Задача 4** 
- Определить автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками

In [ ]:
query = '''
          SELECT ai.author,
                 AVG(ai.avg_rating)
          FROM (SELECT a.author AS author,
                       AVG(r.rating) AS avg_rating
                FROM authors AS a
                JOIN books AS b ON a.author_id = b.author_id
                JOIN ratings AS r ON r.book_id = b.book_id
                GROUP BY a.author, b.book_id
                HAVING COUNT(r.rating) >= 50) AS ai
           GROUP BY ai.author
           ORDER BY AVG(ai.avg_rating) DESC
           LIMIT 1
        '''
pd.io.sql.read_sql(query, con = engine) 

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


"Не жалей умерших, Гарри. Жалей живых, и в особенности тех, кто живет без любви."
<br>Джоан Роулинг и иллюстратор Мари Грандпре заняли первую позицию в наших сердечках.

**Задача 5** 
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [ ]:
query = '''
           SELECT AVG(new.count_review) AS avg_review
           FROM (SELECT COUNT(review_id) AS count_review
                 FROM reviews
                 WHERE username IN (SELECT username
                                    FROM ratings
                                    GROUP BY username
                                    HAVING COUNT(rating_id)>50)
                GROUP BY username) 
          AS new
        '''
pd.io.sql.read_sql(query, con = engine) 

,avg_review
0,24.333333
